Queue, Pipe <br>
Communications between processes

In [1]:
from multiprocessing import Process, Queue, current_process
import os, time
# 프로세스 통신 구현 Queue

def worker(id, base_number, q):
    process_id = os.getpid()
    process_name = current_process().name

    # 누적
    sub_total = 0

    # 계산
    for i in range(base_number):
        sub_total +=1

    # Producer
    q.put(sub_total)

    # 정보출력
    print(f'Process ID: {process_id}, Process name: {process_name}, ID: {id}')
    print(f'Result: {sub_total}')

def main():
    # 부모 프로세스 id
    parent_process_id = os.getpid()
    #출력
    print(f'Parent process ID: {parent_process_id}')

    # 프로세스 리스트 선언
    processes = list()
    # 시작시간
    start_time = time.time()

    # Queue 선언
    q = Queue()

    for i in range(5):
        #생성
        t = Process(name=str(i), target=worker, args=(1,100000000, q))

        #배열에 담기
        processes.append(t)
        # 작업 시작
        t.start()

    for proc in processes:
        proc.join()

    # 순수 계산 시간
    print(f'---{time.time() - start_time} seconds ---')

    # 종료  flag
    q.put('exit')

    total = 0
    # 대기
    while True:
        tmp = q.get()
        if tmp == 'exit':
            break
        else:
            total += tmp
    print()
    print(f'Main-Processing Total Count == {total}')
    print('Main-Processing Done!')


In [2]:
if __name__ == "__main__":
    main()

Parent process ID: 40052
---0.9979987144470215 seconds ---

Main-Processing Total Count == 0
Main-Processing Done!


Pipe: 1 대 1로 정보를 보내는 것이다.  <br>
Queue는 대응을 사용자가 설계할 수 있다. <br>

In [3]:
from multiprocessing import Process, Pipe, current_process
import os, time
# 프로세스 통신 구현 Queue

def worker(id, base_number, conn):
    process_id = os.getpid()
    process_name = current_process().name

    # 누적
    sub_total = 0

    # 계산
    for i in range(base_number):
        sub_total +=1

    # Producer
    conn.send(sub_total)
    conn.close()
    # 1 대 1이기 때문에 한 번 쓰면 다른 곳에서 쓰게끔 잠가야 한다.

    # 정보출력
    print(f'Process ID: {process_id}, Process name: {process_name}, ID: {id}')
    print(f'Result: {sub_total}')

def main():
    # 부모 프로세스 id
    parent_process_id = os.getpid()
    #출력
    print(f'Parent process ID: {parent_process_id}')

    
    # 시작시간
    start_time = time.time()

    # Pipe 선언
    parent_conn, child_conn = Pipe()

    
    #생성
    t = Process(name= 'pipe_ex', target=worker, args=(1,100000000, child_conn))

    
    # 작업 시작
    t.start()

    
    t.join()

    # 순수 계산 시간
    print(f'---{time.time() - start_time} seconds ---')

  
    total = parent_conn.recv()
   
    print()
    print(f'Main-Processing Total Count == {total}')
    print('Main-Processing Done!')

In [4]:
if __name__ == '__main__':
    main()

# 주피터노트북이 안 되고 있고 py file로 실행하면 잘 된다.

Parent process ID: 40052
---0.38099217414855957 seconds ---
